<h1>Table Description Metadata Writer</h1>

This notebook is meant as an entrypoint to write metadata descriptions for tables and columns using ingested `yml` files structured in the following way:
<br>

```
tables:
        - name: foo
            description: Description of the table
            columns:
            - name: column1
                description: Description of column1
            - name: column2
                description: Description of column2
        - name: bar
            description: Description of another table
            columns:
            - name: columnA
                description: Description of columnA
```

The underlying libraries are included, however, if you would like to process a directory of `yml` metadata and apply them to the associated tables in a single `catalog.schema`, fill in the global variables in the cell below and place your metadata in the `./metadata` directory in this repository. Alternatively, you can change the path to the metadata files in the global variables below.

In [ ]:
%pip install pyyaml

In [ ]:
CATALOG = "< your catalog name here >"
SCHEMA = "< your schema name here> "
PATH_TO_METADATA = "./metadata"

In [ ]:
# Register the describe_table function for use by the agentic model
sql = f"""
CREATE OR REPLACE FUNCTION {CATALOG}.tools.describe_table(
  cat STRING COMMENT 'Catalog name',
  sch STRING COMMENT 'Schema name',
  tbl STRING COMMENT 'Table name'
)
RETURNS TABLE (
  column_name STRING COMMENT 'Column name',
  data_type STRING COMMENT 'Data type',
  is_nullable STRING COMMENT 'YES or NO',
  comment STRING COMMENT 'Column comment',
  ordinal_position INT COMMENT '1-based position in table'
)
COMMENT 'Returns column metadata like DESCRIBE for a given <catalog>.<schema>.<table>.'
RETURN
SELECT
  c.column_name,
  c.data_type,
  c.is_nullable,
  c.comment,
  c.ordinal_position
FROM system.information_schema.columns AS c
WHERE c.table_catalog = cat
  AND c.table_schema  = sch
  AND c.table_name    = tbl
ORDER BY c.ordinal_position;
"""

spark.sql(sql)

In [ ]:
from utils.metadata_processor import process_metadata

process_metadata(PATH_TO_METADATA, CATALOG, SCHEMA)